In [20]:
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from tensorflow import keras
from graphing_functions import *
from data_preprocessing import *
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Read stock prices data
stock_price_df = pd.read_csv("stock.csv")
stock_price_df

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,60.198570,75.510002,30.120001,12.130000,175.929993,180.550003,28.250000,313.644379,1295.500000
1,2012-01-13,59.972858,74.599998,30.070000,12.350000,178.419998,179.160004,22.790001,311.328064,1289.089966
2,2012-01-17,60.671429,75.239998,30.250000,12.250000,181.660004,180.000000,26.600000,313.116364,1293.670044
3,2012-01-18,61.301430,75.059998,30.330000,12.730000,189.440002,181.070007,26.809999,315.273285,1308.040039
4,2012-01-19,61.107143,75.559998,30.420000,12.800000,194.449997,180.520004,26.760000,318.590851,1314.500000
...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,440.250000,174.279999,29.850000,16.719999,3205.030029,125.449997,1485.020020,1473.609985,3327.770020
2155,2020-08-06,455.609985,172.199997,29.840000,18.459999,3225.000000,126.120003,1489.579956,1500.099976,3349.159912
2156,2020-08-07,444.450012,170.020004,30.020000,19.030001,3167.459961,124.959999,1452.709961,1494.489990,3351.280029
2157,2020-08-10,450.910004,179.410004,30.200001,21.650000,3148.159912,127.110001,1418.569946,1496.099976,3360.469971


In [3]:
# Read the stocks volume data
stock_vol_df = pd.read_csv("stock_volume.csv")
stock_vol_df

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,53146800,3934500,26511100,17891100,5385800,6881000,729300,3764400,4019890000
1,2012-01-13,56505400,4641100,22096800,16621800,4753500,5279200,5500400,4631800,3692370000
2,2012-01-17,60724300,3700100,23500200,15480800,5644500,6003400,4651600,3832800,4010490000
3,2012-01-18,69197800,4189500,22015000,18387600,7473500,4600600,1260200,5544000,4096160000
4,2012-01-19,65434600,5397300,25524000,14022900,7096000,8567200,1246300,12657800,4465890000
...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,30498000,46551000,22991700,18914200,3930000,3675400,4978000,1979500,4732220000
2155,2020-08-06,50607200,32921600,21908700,35867700,3940600,3417100,5992300,1995400,4267490000
2156,2020-08-07,49453300,19301600,30398500,34530300,3929600,3651000,8883500,1576600,4104860000
2157,2020-08-10,53100900,35857700,35514400,71219700,3167300,3968300,7522300,1289300,4318570000


In [4]:
stock_price_df,stats = read_dataset(stock_price_df)
stats

DataFrame sorted by Dates in ascending order
No null values
number of stocks is : 9
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2159 entries, 0 to 2158
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2159 non-null   object 
 1   AAPL    2159 non-null   float64
 2   BA      2159 non-null   float64
 3   T       2159 non-null   float64
 4   MGM     2159 non-null   float64
 5   AMZN    2159 non-null   float64
 6   IBM     2159 non-null   float64
 7   TSLA    2159 non-null   float64
 8   GOOG    2159 non-null   float64
 9   sp500   2159 non-null   float64
dtypes: float64(9), object(1)
memory usage: 185.5+ KB


,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
count,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000
mean,140.819823,189.942700,35.162899,23.105743,915.665665,161.853001,259.600815,783.712512,2218.749554
std,70.827601,103.678586,3.207490,6.963847,697.838905,25.561938,210.988003,334.448057,537.321727
min,55.790001,67.239998,26.770000,7.140000,175.929993,94.769997,22.790001,278.481171,1278.040039
25%,89.165714,124.015000,33.040001,18.545000,316.490005,142.769997,184.595001,527.214416,1847.984985
50%,116.599998,142.419998,34.930000,23.780001,676.010010,156.949997,231.960007,737.599976,2106.629883
75%,175.019997,297.044998,37.419998,28.430000,1593.645019,185.974998,307.350006,1079.744995,2705.810059
max,455.609985,440.619995,43.470001,38.029999,3225.000000,215.800003,1643.000000,1568.489990,3386.149902


In [5]:
stock_vol_df,stats = read_dataset(stock_vol_df)
stats

DataFrame sorted by Dates in ascending order
No null values
number of stocks is : 9
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2159 entries, 0 to 2158
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    2159 non-null   object
 1   AAPL    2159 non-null   int64 
 2   BA      2159 non-null   int64 
 3   T       2159 non-null   int64 
 4   MGM     2159 non-null   int64 
 5   AMZN    2159 non-null   int64 
 6   IBM     2159 non-null   int64 
 7   TSLA    2159 non-null   int64 
 8   GOOG    2159 non-null   int64 
 9   sp500   2159 non-null   int64 
dtypes: int64(9), object(1)
memory usage: 185.5+ KB


,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
count,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03
mean,5.820332e+07,6.419916e+06,2.832131e+07,9.845582e+06,4.102673e+06,4.453090e+06,7.001302e+06,2.498238e+06,3.680732e+09
std,4.568141e+07,9.711873e+06,1.428911e+07,7.295753e+06,2.290722e+06,2.462811e+06,5.781208e+06,1.928407e+06,8.622717e+08
min,1.136200e+07,7.889000e+05,6.862400e+06,9.507000e+05,8.813000e+05,1.193000e+06,3.649000e+05,7.900000e+03,1.248960e+09
25%,2.769930e+07,3.031850e+06,2.002150e+07,5.796450e+06,2.675700e+06,3.111250e+06,3.433450e+06,1.325400e+06,3.211890e+09
50%,4.209420e+07,3.991000e+06,2.485930e+07,7.899800e+06,3.494800e+06,3.825000e+06,5.581100e+06,1.813900e+06,3.526890e+09
75%,7.182480e+07,5.325900e+06,3.210565e+07,1.104055e+07,4.768150e+06,4.937300e+06,8.619550e+06,3.245350e+06,3.933290e+09
max,3.765300e+08,1.032128e+08,1.950827e+08,9.009820e+07,2.385610e+07,3.049020e+07,6.093880e+07,2.497790e+07,9.044690e+09


# TASK #4: PREPARE THE DATA BEFORE TRAINING THE AI/ML MODEL

In [11]:
prediction_df,price_vol_target_df =  create_stock_df(stock_price_df,stock_vol_df,"AAPL",5)


In [12]:

price_vol_target_df

,date,close,volume,target
0,2012-01-12,60.198570,53146800,60.042858
1,2012-01-13,59.972858,56505400,61.058571
2,2012-01-17,60.671429,60724300,60.058571
3,2012-01-18,61.301430,69197800,63.808571
4,2012-01-19,61.107143,65434600,63.518570
...,...,...,...,...
2149,2020-07-29,380.160004,22582300,440.250000
2150,2020-07-30,384.760010,39532500,455.609985
2151,2020-07-31,425.040009,93584200,444.450012
2152,2020-08-03,435.750000,77037800,450.910004


In [13]:
prediction_df

,date,close,volume,target
2154,2020-08-05,440.250000,30498000,NaN
2155,2020-08-06,455.609985,50607200,NaN
2156,2020-08-07,444.450012,49453300,NaN
2157,2020-08-10,450.910004,53100900,NaN
2158,2020-08-11,437.500000,46871100,NaN


# TASK #12: TRAIN AN LSTM TIME SERIES MODEL

In [16]:
def transform_data(df):
  sc_X = MinMaxScaler(feature_range = (0,1))
  sc_y = MinMaxScaler(feature_range = (0,1))
  X = df.iloc[:, 1:-1].values
  y = df.iloc[:, -1].values
  y = y.reshape(len(y),1)
  X = sc_X.fit_transform(X)
  y = sc_y.fit_transform(y) 
  return sc_X,sc_y,X,y

In [18]:
# Get the close and volume data as training data (Input)
price_vol_target_df

,date,close,volume,target
0,2012-01-12,60.198570,53146800,60.042858
1,2012-01-13,59.972858,56505400,61.058571
2,2012-01-17,60.671429,60724300,60.058571
3,2012-01-18,61.301430,69197800,63.808571
4,2012-01-19,61.107143,65434600,63.518570
...,...,...,...,...
2149,2020-07-29,380.160004,22582300,440.250000
2150,2020-07-30,384.760010,39532500,455.609985
2151,2020-07-31,425.040009,93584200,444.450012
2152,2020-08-03,435.750000,77037800,450.910004


In [36]:
sc_X,sc_y,X,y = transform_data(price_vol_target_df)

In [39]:
X

array([[0.01151453, 0.11442624],
       [0.01092501, 0.12362365],
       [0.01274957, 0.13517696],
       ...,
       [0.96442658, 0.22516266],
       [0.9923995 , 0.17985092],
       [1.        , 0.08737321]])

In [22]:
# Convert the data into array format
X,y = np.asarray(X),np.asarray(y)

In [26]:
# Split the data
X_train,y_train,X_test,y_test = split_timeseries(X,y)

In [27]:
# Reshape the 1D arrays to 3D arrays to feed in the model
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
X_train.shape,X_test.shape

((1723, 2, 1), (431, 2, 1))

In [31]:
# Create the model
inputs = keras.layers.Input(shape = (X_train.shape[1],X_train.shape[2]))
x = keras.layers.LSTM(150, return_sequences=True)(inputs)
x = keras.layers.LSTM(150, return_sequences=True)(x)
x = keras.layers.LSTM(150, return_sequences=True)(x)
outputs = keras.layers.Dense(1, activation= "linear")(x)

model = keras.Model(inputs = inputs , outputs = outputs)
model.compile(optimizer = "adam",loss = "mse")
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2, 1)]            0         
_________________________________________________________________
lstm (LSTM)                  (None, 2, 150)            91200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 150)            180600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 2, 150)            180600    
_________________________________________________________________
dense (Dense)                (None, 2, 1)              151       
Total params: 452,551
Trainable params: 452,551
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Train the model
history = model.fit(X_train,y_train,epochs = 20,batch_size=32,validation_split=0.2)

Epoch 1/20
44/44 [==============================] - 2s 35ms/step - loss: 0.0052 - val_loss: 0.0430
Epoch 2/20
44/44 [==============================] - 0s 11ms/step - loss: 0.0033 - val_loss: 0.0277
Epoch 3/20
44/44 [==============================] - 1s 11ms/step - loss: 0.0020 - val_loss: 0.0127
Epoch 4/20
44/44 [==============================] - 0s 11ms/step - loss: 0.0014 - val_loss: 0.0137
Epoch 5/20
44/44 [==============================] - 1s 12ms/step - loss: 0.0014 - val_loss: 0.0139
Epoch 6/20
44/44 [==============================] - 0s 10ms/step - loss: 0.0014 - val_loss: 0.0139
Epoch 7/20
44/44 [==============================] - 0s 10ms/step - loss: 0.0014 - val_loss: 0.0137
Epoch 8/20
44/44 [==============================] - 1s 12ms/step - loss: 0.0014 - val_loss: 0.0150
Epoch 9/20
44/44 [==============================] - 0s 10ms/step - loss: 0.0014 - val_loss: 0.0147
Epoch 10/20
44/44 [==============================] - 1s 12ms/step - loss: 0.0013 - val_loss: 0.0135
Epoch 11/

In [35]:
# Make prediction
predicted = model.predict(X)
predicted

array([[[0.01174727],
        [0.00384014]],

       [[0.0110901 ],
        [0.00439945]],

       [[0.01312207],
        [0.00771214]],

       ...,

       [[0.4195749 ],
        [0.58039784]],

       [[0.41900507],
        [0.59639585]],

       [[0.41879058],
        [0.6193053 ]]], dtype=float32)

In [34]:
# Append the predicted values to a list
test_predicted = [ i[0][0] for i in predicted]
close = [i[0] for i in scaled_training]

NameError: name 'scaled_training' is not defined

In [62]:
df_predicted["close"] = close
df_predicted["Date"] = price_vol_df["date"]
df_predicted['predictions'] = test_predicted

ValueError: Length of values (2159) does not match length of index (2158)

In [118]:
df_predicted

,date,close,predictions,Date
1,2012-01-13,0.011026,0.010077,2012-01-13
2,2012-01-17,0.010462,0.009525,2012-01-17
3,2012-01-18,0.012209,0.011233,2012-01-18
4,2012-01-19,0.013785,0.012773,2012-01-19
5,2012-01-20,0.013299,0.012298,2012-01-20
...,...,...,...,...
2154,2020-08-05,0.957606,0.945752,2020-08-05
2155,2020-08-06,0.961583,0.949527,2020-08-06
2156,2020-08-07,1.000000,0.985843,2020-08-07
2157,2020-08-10,0.972088,0.959485,2020-08-10


In [109]:
|df_predicted["predictions"] = test_predicted

In [110]:
df_predicted

,date,close,predictions
1,2012-01-13,59.972858,0.010077
2,2012-01-17,60.671429,0.009525
3,2012-01-18,61.301430,0.011233
4,2012-01-19,61.107143,0.012773
5,2012-01-20,60.042858,0.012298
...,...,...,...
2154,2020-08-05,440.250000,0.945752
2155,2020-08-06,455.609985,0.949527
2156,2020-08-07,444.450012,0.985843
2157,2020-08-10,450.910004,0.959485


In [119]:
interactive_plot(df_predicted,"ORIGINAL VS PREDICTIONS : LSTM")

**MINI CHALLENGE #5:**
- **Test the pipeline with at least 3 other stocks** 
- **Experiment with various LSTM model parameters (Ex: Use 150 units instead of 50), print out the model summary and retrain the model**


# **EXCELLENT JOB! YOU SHOULD BE TRULY PROUD WITH THE NEWLY ACQUIRED SKILLS**

# **MINI CHALLENGE SOLUTIONS**

**MINI CHALLENGE #1 SOLUTION:**
- **What is the average trading volume for Apple stock?**
- **What is the maximum trading volume for sp500?**
- **Which security is traded the most? comment on your answer**
- **What is the average stock price of the S&P500 over the specified time period?**
- **What is the maximum price of Tesla Stock?**

In [61]:
# Get the statistical data for the stocks volume dataframe
# Average trading volume for Apple stock is 2.498238e+06	
# Average trading volume for S&P500 is 3.680732e+09

# Why S&P500 is the most traded of all? Check out this article: 
# https://www.investopedia.com/articles/personal-finance/022216/put-10000-sp-500-etf-and-wait-20-years.asp

# Text from the article above:
# "The S&P 500 index is a broad-based measure of large corporations traded on U.S. stock markets. Over long periods of time, 
# passively holding the index often produces better results than actively trading or picking single stocks.
# Over long-time horizons, the index typically produces better returns than actively managed portfolios."

stock_vol_df.describe()

,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
count,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03
mean,5.820332e+07,6.419916e+06,2.832131e+07,9.845582e+06,4.102673e+06,4.453090e+06,7.001302e+06,2.498238e+06,3.680732e+09
std,4.568141e+07,9.711873e+06,1.428911e+07,7.295753e+06,2.290722e+06,2.462811e+06,5.781208e+06,1.928407e+06,8.622717e+08
min,1.136200e+07,7.889000e+05,6.862400e+06,9.507000e+05,8.813000e+05,1.193000e+06,3.649000e+05,7.900000e+03,1.248960e+09
25%,2.769930e+07,3.031850e+06,2.002150e+07,5.796450e+06,2.675700e+06,3.111250e+06,3.433450e+06,1.325400e+06,3.211890e+09
50%,4.209420e+07,3.991000e+06,2.485930e+07,7.899800e+06,3.494800e+06,3.825000e+06,5.581100e+06,1.813900e+06,3.526890e+09
75%,7.182480e+07,5.325900e+06,3.210565e+07,1.104055e+07,4.768150e+06,4.937300e+06,8.619550e+06,3.245350e+06,3.933290e+09
max,3.765300e+08,1.032128e+08,1.950827e+08,9.009820e+07,2.385610e+07,3.049020e+07,6.093880e+07,2.497790e+07,9.044690e+09


In [62]:
# Get the statistical data for the prices dataframe
stock_price_df.describe()

,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
count,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000
mean,140.819823,189.942700,35.162899,23.105743,915.665665,161.853001,259.600815,783.712512,2218.749554
std,70.827601,103.678586,3.207490,6.963847,697.838905,25.561938,210.988003,334.448057,537.321727
min,55.790001,67.239998,26.770000,7.140000,175.929993,94.769997,22.790001,278.481171,1278.040039
25%,89.165714,124.015000,33.040001,18.545000,316.490005,142.769997,184.595001,527.214416,1847.984985
50%,116.599998,142.419998,34.930000,23.780001,676.010010,156.949997,231.960007,737.599976,2106.629883
75%,175.019997,297.044998,37.419998,28.430000,1593.645019,185.974998,307.350006,1079.744995,2705.810059
max,455.609985,440.619995,43.470001,38.029999,3225.000000,215.800003,1643.000000,1568.489990,3386.149902


In [63]:
# Average price for S&P500 = 2218.749554
# Maximum Tesla Price = 1643.000000

**MINI CHALLENGE #2 SOLUTION:**
- **Plot the normalized stock prices and volume dataset.**

In [64]:
# Plot interactive chart for volume data
# Notice that S&P500 trading is orders of magnitude compared to individual stocks
interactive_plot(stock_vol_df, 'Stocks Volume')

In [65]:
# plot interactive chart for normalized stocks prices data
interactive_plot(normalize(stock_price_df), 'Stock Prices')

# Let's normalize the data and re-plot interactive chart for volume data
interactive_plot(normalize(stock_vol_df), 'Normalized Volume')

**MINI CHALLENGE #3 SOLUTION:**
- **Test the pipeline with S&P500 and AMZN datasets instead of AAPL**

In [66]:
# Let's test the functions and get individual stock prices and volumes for S&P500
price_volume_df = individual_stock(stock_price_df, stock_vol_df, 'sp500')
price_volume_df

# Let's test the functions and get individual stock prices and volumes for Amazon 
price_volume_df = individual_stock(stock_price_df, stock_vol_df, 'AMZN')
price_volume_df

,date,close,volume
0,2012-01-12,175.929993,5385800
1,2012-01-13,178.419998,4753500
2,2012-01-17,181.660004,5644500
3,2012-01-18,189.440002,7473500
4,2012-01-19,194.449997,7096000
...,...,...,...
2154,2020-08-05,3205.030029,3930000
2155,2020-08-06,3225.000000,3940600
2156,2020-08-07,3167.459961,3929600
2157,2020-08-10,3148.159912,3167300


**MINI CHALLENGE #4 SOLUTION:**
- **Experiment with various regularization value for alpha** 
- **What is the impact of increasing alpha?**
- **Note: default value for alpha is = 1**

In [67]:
from sklearn.linear_model import Ridge
# Note that Ridge regression performs linear least squares with L2 regularization.
# Create and train the Ridge Linear Regression  Model
regression_model = Ridge(alpha = 2)
regression_model.fit(X_train, y_train)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

**MINI CHALLENGE #5 SOLUTION:**
- **Test the pipeline with at least 3 other stocks** 
- **Experiment with various LSTM model parameters (Ex: Use 150 units instead of 50), print out the model summary and retrain the model**



In [68]:
# Create the model
inputs = keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
x = keras.layers.LSTM(150, return_sequences= True)(inputs)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.LSTM(150, return_sequences=True)(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.LSTM(150)(x)
outputs = keras.layers.Dense(1, activation='linear')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss="mse")
model.summary()

IndexError: ignored